In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth

#Spotipy documentation: https://spotipy.readthedocs.io/en/master/

In [9]:
#Write here the client ID and secret ID from spotify API dashboard
SPOTIPY_CLIENT_ID = ''
SPOTIPY_CLIENT_SECRET = ''
REDIRECT_URI = 'http://localhost:7000/callback'
scope = "user-library-read"

cache_handler = spotipy.cache_handler.MemoryCacheHandler()
auth_manager = SpotifyClientCredentials(client_id = SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET, cache_handler=cache_handler)

sp = spotipy.Spotify(auth_manager = auth_manager)

In [10]:
#Testing finding genre
results = sp.available_markets()
sp.category('0JQ5DAqbMKFEC4WFtoNRpw', country = 'FI')
pop = '0JQ5DAqbMKFEC4WFtoNRpw'

Next cell below finds a random song in the finnish market according to spotify, returns the name, popularity and audio features

In [32]:
import random
import string
random_character = random.choice(string.ascii_letters)
random_search = random_character + '%'
offset = random.randint(1,1000)
year = '1990'

songs = sp.search(q = 'track:' + random_search + ' year:' + year, type = 'track', market = 'FI', offset = offset, limit = 1)

first_song = songs['tracks']['items'][0]
first_song_name = first_song['name']
first_song_popularity = first_song['popularity']
first_song_audio_features = sp.audio_features(first_song['id'])


first_song_name, first_song_popularity, first_song_audio_features

#features = sp.audio_features(id)
#features

('Taloni sanoo minulle',
 2,
 [{'danceability': 0.421,
   'energy': 0.115,
   'key': 10,
   'loudness': -16.429,
   'mode': 1,
   'speechiness': 0.0398,
   'acousticness': 0.982,
   'instrumentalness': 0.131,
   'liveness': 0.114,
   'valence': 0.11,
   'tempo': 65.717,
   'type': 'audio_features',
   'id': '1HewnZSHNxPSBAKBri30Yg',
   'uri': 'spotify:track:1HewnZSHNxPSBAKBri30Yg',
   'track_href': 'https://api.spotify.com/v1/tracks/1HewnZSHNxPSBAKBri30Yg',
   'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1HewnZSHNxPSBAKBri30Yg',
   'duration_ms': 244706,
   'time_signature': 4}])